# Select the selection algorithms

The idea here is that we can simplify the decision logic, reduce the binary size
and speed up the compilation time by only including a subset of selection algorithms.
We're aiming to get algorithms that perform well in different situations, and complement
each other - so to do this, we're iteratively removing the worst performing algorithm,
after which algorithms are re-evaluated on their speedups relative to the remaining
algorithms. This gets us a minimum spanning set of selection algorithms that performs
well over diverse inputs.

In [1]:
from select_k_dataset import load_dataframe, get_dataset

df = load_dataframe("select_k_times.json")
df

,key_type,index_type,algo,row,col,k,use_index_input,use_memory_pool,time
0,double,int64_t,kRadix8bits,1,1024,1,0,0,0.000050
1,double,int64_t,kRadix11bits,1,1024,1,0,0,0.000033
2,double,int64_t,kRadix11bitsExtraPass,1,1024,1,0,0,0.000033
3,double,int64_t,kWarpImmediate,1,1024,1,0,0,0.000022
4,double,int64_t,kWarpFiltered,1,1024,1,0,0,0.000024
...,...,...,...,...,...,...,...,...,...
179963,float,uint32_t,kRadix11bits,1075,2042,8175,0,0,0.001018
179964,float,uint32_t,kRadix11bitsExtraPass,1075,2042,8175,0,0,0.001018
179965,float,uint32_t,kRadix8bits,1075,2042,8175,0,1,0.000059
179966,float,uint32_t,kRadix11bits,1075,2042,8175,0,1,0.000072


In [2]:
from collections import Counter

def rank_algos(df, use_relative_speedup=False):
    _, y, weights = get_dataset(df)
    times = Counter()
    for algo, speedup in zip(y, weights):
        times[algo] += speedup if use_relative_speedup else 1
    return sorted(times.items(), key=lambda x:-x[-1])

In [3]:
# show the number of times each algorithm is fastest for a given k/# of rows/# of cols / dtype / memory pool etc
rank_algos(df)

[('kRadix11bits', 7267),
 ('kWarpDistributedShm', 6861),
 ('kFaissBlockSelect', 3620),
 ('kRadix8bits', 3229),
 ('kWarpDistributed', 2619),
 ('kWarpImmediate', 2584),
 ('kRadix11bitsExtraPass', 2260),
 ('kWarpFiltered', 490)]

In [7]:
# kRadix8bits seems to have a performance issue with 64 bit index types, it is one
# of the worst performing algorithms for 64bit indices, but one of the top 3 for 32 bit
rank_algos(df[df.index_type == "int64_t"])

[('kRadix11bits', 3591),
 ('kWarpDistributedShm', 3589),
 ('kFaissBlockSelect', 2006),
 ('kWarpImmediate', 1552),
 ('kWarpDistributed', 1448),
 ('kRadix11bitsExtraPass', 1338),
 ('kRadix8bits', 460),
 ('kWarpFiltered', 290)]

In [8]:
rank_algos(df[df.index_type == "uint32_t"])

[('kRadix11bits', 3676),
 ('kWarpDistributedShm', 3272),
 ('kRadix8bits', 2769),
 ('kFaissBlockSelect', 1614),
 ('kWarpDistributed', 1171),
 ('kWarpImmediate', 1032),
 ('kRadix11bitsExtraPass', 922),
 ('kWarpFiltered', 200)]

In [19]:
# do an algorithm selection pass, repeatedly remove the lowest performing algorithm
#
# The idea here is that we can simplify the decision logic, reduce the binary size
# and speed up the compilation time by only including a subset of selection algorithms.
# we're aiming to get algorithms that perform well in different situations, and complement
# each other - so to do this, we're iteratively removing the worst performing algorithm,
# after which algorithms are re-evaluated on their speedups relative to the remaining
# algorithms. This gets us a minimum spanning set of selection algorithms that performs
# well over diverse inputs.
#
# note: the lowest performing algorithm here might actually be pretty good, but
# just not provide much benefit over another similar algorithm. 
# As an example, kWarpDistributed  is an excellent selection algorithm, but in testing 
# kWarpDistributedShm is slightly faster than it in situations where it does well, 
# meaning that it gets removed early on in this loop
current = df[df.use_memory_pool == True]
algos = set(df.algo)

# we're arbitrarily getting this down to 3 selection algorithms
while len(algos) > 3:
    times = rank_algos(current, use_relative_speedup=False)
    algo, speedup = times[-1]
    algos.remove(algo)
    current = df[df.algo.isin(algos)]

print("selected", algos)
rank_algos(current)

selected {'kRadix11bits', 'kWarpDistributedShm', 'kFaissBlockSelect'}


[('kRadix11bits', 12736),
 ('kWarpDistributedShm', 12317),
 ('kFaissBlockSelect', 3877)]

In [ ]:
# experimenting with different subsets of index type / dtype / use memory seems
# to pretty consistently show that kRadix11bits / kWarpDistributedShm / kFaissBlockSelect
# all get selected here